## Bus Analysis


In [1]:
import xml.etree.ElementTree as ET
import csv
import os
import datetime
import pandas as pd
import glob

In [2]:
def importJourneyDataToCSV(_fileAddress):
    print("File Address = " + _fileAddress)

    if "BusData_" not in _fileAddress:
        return -1

    tree = ET.parse(_fileAddress)
    root = tree.getroot()
    
    fileName = _fileAddress.split("/")[-1]

    if root.tag != "Siri":
        print("{0}is not a correct Data file".format(_fileAddress))
        return 0
    #root[0] is <Element 'ServiceDelivery' at 0x109773278>
    ServiceDelivery=root[0]
    if(ServiceDelivery.tag != "ServiceDelivery"):
        print("Var ServiceDelivery = " + ServiceDelivery.tag)

    #root[0][0] is <Element 'ResponseTimestamp' at 0x1097732c8>
    ResponseTimestamp = root[0][0]
    if(ResponseTimestamp.tag !="ResponseTimestamp"):
        print("Var ResponseTimestamp = " + ResponseTimestamp.tag)

    #root[0][1] is "<Element 'VehicleMonitoringDelivery' at 0x1097733b8>"
    VehicleMonitoringDelivery = root[0][1]
    if(VehicleMonitoringDelivery.tag != "VehicleMonitoringDelivery"):
        print("Var VehicleMonitoringDelivery = " + VehicleMonitoringDelivery.tag)

    #root[0][2] is "<Element 'SituationExchangeDelivery' at 0x10a835278>"
    SituationExchangeDelivery = root[0][2]
    if(SituationExchangeDelivery.tag != "SituationExchangeDelivery"):
        print("Var SituationExchangeDelivery = " + SituationExchangeDelivery.tag)
        
    df_ = pd.DataFrame( columns=['LineRef','DirectionRef', 'DatedVehicleJourneyRef', 'DataFrameRef', 'JourneyPatternRef', 
                            'OriginRef', 'DestinationRef', 'DestinationName', 'RecordedAtTime', 'Latitude', 'Longitude', 'Bearing', 'ProgressRate', 'progressStatus', 'fileName'] )
        
    #Read vehicles' position and time 
    for VehicleActivity in VehicleMonitoringDelivery.findall("VehicleActivity"):

        #Get Line Ref
        for MonitoredVehicleJourney in VehicleActivity.findall("MonitoredVehicleJourney"):

            # Line Name
            lineRef = MonitoredVehicleJourney.find("LineRef").text
            directionRef = MonitoredVehicleJourney.find("DirectionRef").text
            journeyRef = MonitoredVehicleJourney.find("FramedVehicleJourneyRef/DatedVehicleJourneyRef").text
            dateFrameRef = MonitoredVehicleJourney.find("FramedVehicleJourneyRef/DataFrameRef").text

        
            pattern = MonitoredVehicleJourney.find("JourneyPatternRef").text
            origin = MonitoredVehicleJourney.find("OriginRef").text
            destination = MonitoredVehicleJourney.find("DestinationRef").text
            destinationName = MonitoredVehicleJourney.find("DestinationName").text            
            
            
            timeText = VehicleActivity.find("RecordedAtTime").text
            lat = MonitoredVehicleJourney.find("VehicleLocation/Latitude").text
            lon = MonitoredVehicleJourney.find("VehicleLocation/Longitude").text
            bearing = MonitoredVehicleJourney.find("Bearing").text
            progressRate = MonitoredVehicleJourney.find("ProgressRate").text
            progressStatus = "None"
            if MonitoredVehicleJourney.find("ProgressStatus") is None:
                pass
            else:
                progressStatus = MonitoredVehicleJourney.find("ProgressStatus").text

            
            tmp_se = pd.Series( [lineRef, directionRef, journeyRef, dateFrameRef, pattern, origin, destination, destinationName, 
                                 timeText, lat, lon, bearing, progressRate, progressStatus, fileName], index=df_.columns )    
            df_ = df_.append( tmp_se, ignore_index=True )
    
    newFile = _fileAddress.replace("xml", "csv")
    df_.to_csv(newFile, index=False)

In [3]:
filelists = glob.glob('data/xml/20190212/BusData_20190212_07*.xml')
filelists += glob.glob('data/xml/20190212/BusData_20190212_08*.xml')
filelists += glob.glob('data/xml/20190212/BusData_20190212_09*.xml')

In [4]:
for fileName in filelists:
    importJourneyDataToCSV(fileName)

File Address = data/xml/20190212/BusData_20190212_072200.xml
File Address = data/xml/20190212/BusData_20190212_070600.xml
File Address = data/xml/20190212/BusData_20190212_070400.xml
File Address = data/xml/20190212/BusData_20190212_075800.xml
File Address = data/xml/20190212/BusData_20190212_072000.xml
File Address = data/xml/20190212/BusData_20190212_072400.xml
File Address = data/xml/20190212/BusData_20190212_070000.xml
File Address = data/xml/20190212/BusData_20190212_070200.xml
File Address = data/xml/20190212/BusData_20190212_072600.xml
File Address = data/xml/20190212/BusData_20190212_073800.xml
File Address = data/xml/20190212/BusData_20190212_074000.xml
File Address = data/xml/20190212/BusData_20190212_074200.xml
File Address = data/xml/20190212/BusData_20190212_074600.xml
File Address = data/xml/20190212/BusData_20190212_071800.xml
File Address = data/xml/20190212/BusData_20190212_074400.xml
File Address = data/xml/20190212/BusData_20190212_074800.xml
File Address = data/xml/

In [5]:
csvfilelists = glob.glob('data/csv/20190212/*.csv')

In [6]:
df_all = None
for file in csvfilelists:
    #print(file)
    if df_all is None:
        df_all= pd.read_csv(file)
    else:
        df_all = pd.concat([df_all,pd.read_csv(file)])

In [7]:
df_all.head()

,LineRef,DirectionRef,DatedVehicleJourneyRef,DataFrameRef,JourneyPatternRef,OriginRef,DestinationRef,DestinationName,RecordedAtTime,Latitude,Longitude,Bearing,ProgressRate,progressStatus,fileName
0,MTA NYCT_M55,0,MTA NYCT_MQ_A9-Weekday-SDon-049500_M55_3,2019-02-12,MTA_M550034,MTA_803184,MTA_803081,44 ST 6 AV,2019-02-12T08:45:33.000-05:00,40.743435,-73.992396,52.73974,normalProgress,None,BusData_20190212_084600.xml
1,MTA NYCT_M86+,1,MTA NYCT_MQ_A9-Weekday-SDon-050200_SBS86_19,2019-02-12,MTA_SBS860049,MTA_405003,MTA_803043,SELECT BUS WESTSIDE WEST END AV CROSSTWN,2019-02-12T08:45:53.000-05:00,40.788189,-73.976238,157.56291,normalProgress,None,BusData_20190212_084600.xml
2,MTA NYCT_M55,1,MTA NYCT_MQ_A9-Weekday-SDon-048500_M55_4,2019-02-12,MTA_M550035,MTA_803080,MTA_803185,SOUTH FERRY,2019-02-12T08:45:34.000-05:00,40.702345,-74.013804,327.09476,normalProgress,None,BusData_20190212_084600.xml
3,MTA NYCT_M42,0,MTA NYCT_MQ_A9-Weekday-SDon-051400_M42_11,2019-02-12,MTA_M420133,MTA_803055,MTA_903046,UN - 1 AV CROSSTOWN,2019-02-12T08:45:28.000-05:00,40.760944,-73.998622,336.37064,normalProgress,None,BusData_20190212_084600.xml
4,MTA NYCT_M79+,0,MTA NYCT_MQ_A9-Weekday-SDon-051900_SB79_11,2019-02-12,MTA_SB790011,MTA_403522,MTA_401878,SELECT BUS YRKVLLE EAST END AV CROSSTOWN,2019-02-12T08:45:49.000-05:00,40.783061,-73.974065,336.02057,normalProgress,None,BusData_20190212_084600.xml


In [8]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 309744 entries, 0 to 3076
Data columns (total 15 columns):
LineRef                   309744 non-null object
DirectionRef              309744 non-null int64
DatedVehicleJourneyRef    309744 non-null object
DataFrameRef              309744 non-null object
JourneyPatternRef         309744 non-null object
OriginRef                 309744 non-null object
DestinationRef            309744 non-null object
DestinationName           309744 non-null object
RecordedAtTime            309744 non-null object
Latitude                  309744 non-null float64
Longitude                 309744 non-null float64
Bearing                   309744 non-null float64
ProgressRate              309744 non-null object
progressStatus            309744 non-null object
fileName                  309744 non-null object
dtypes: float64(3), int64(1), object(11)
memory usage: 37.8+ MB


In [9]:
df_all["time_dt"] = pd.to_datetime(df_all["RecordedAtTime"])

In [10]:
df_all.head()

,LineRef,DirectionRef,DatedVehicleJourneyRef,DataFrameRef,JourneyPatternRef,OriginRef,DestinationRef,DestinationName,RecordedAtTime,Latitude,Longitude,Bearing,ProgressRate,progressStatus,fileName,time_dt
0,MTA NYCT_M55,0,MTA NYCT_MQ_A9-Weekday-SDon-049500_M55_3,2019-02-12,MTA_M550034,MTA_803184,MTA_803081,44 ST 6 AV,2019-02-12T08:45:33.000-05:00,40.743435,-73.992396,52.73974,normalProgress,None,BusData_20190212_084600.xml,2019-02-12 08:45:33-05:00
1,MTA NYCT_M86+,1,MTA NYCT_MQ_A9-Weekday-SDon-050200_SBS86_19,2019-02-12,MTA_SBS860049,MTA_405003,MTA_803043,SELECT BUS WESTSIDE WEST END AV CROSSTWN,2019-02-12T08:45:53.000-05:00,40.788189,-73.976238,157.56291,normalProgress,None,BusData_20190212_084600.xml,2019-02-12 08:45:53-05:00
2,MTA NYCT_M55,1,MTA NYCT_MQ_A9-Weekday-SDon-048500_M55_4,2019-02-12,MTA_M550035,MTA_803080,MTA_803185,SOUTH FERRY,2019-02-12T08:45:34.000-05:00,40.702345,-74.013804,327.09476,normalProgress,None,BusData_20190212_084600.xml,2019-02-12 08:45:34-05:00
3,MTA NYCT_M42,0,MTA NYCT_MQ_A9-Weekday-SDon-051400_M42_11,2019-02-12,MTA_M420133,MTA_803055,MTA_903046,UN - 1 AV CROSSTOWN,2019-02-12T08:45:28.000-05:00,40.760944,-73.998622,336.37064,normalProgress,None,BusData_20190212_084600.xml,2019-02-12 08:45:28-05:00
4,MTA NYCT_M79+,0,MTA NYCT_MQ_A9-Weekday-SDon-051900_SB79_11,2019-02-12,MTA_SB790011,MTA_403522,MTA_401878,SELECT BUS YRKVLLE EAST END AV CROSSTOWN,2019-02-12T08:45:49.000-05:00,40.783061,-73.974065,336.02057,normalProgress,None,BusData_20190212_084600.xml,2019-02-12 08:45:49-05:00


#### Get the data from 7:30 am to 9:30 am

In [11]:
df_selected  = df_all[(df_all['time_dt'] >= '2019-2-12 07:30:00') & (df_all['time_dt'] <= '2019-2-12 09:30:00')]

In [12]:
df_selected = df_selected.sort_values(by=['RecordedAtTime'])
df_selected = df_selected.reset_index().drop("index", axis = 1)

In [13]:
df_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 208279 entries, 0 to 208278
Data columns (total 16 columns):
LineRef                   208279 non-null object
DirectionRef              208279 non-null int64
DatedVehicleJourneyRef    208279 non-null object
DataFrameRef              208279 non-null object
JourneyPatternRef         208279 non-null object
OriginRef                 208279 non-null object
DestinationRef            208279 non-null object
DestinationName           208279 non-null object
RecordedAtTime            208279 non-null object
Latitude                  208279 non-null float64
Longitude                 208279 non-null float64
Bearing                   208279 non-null float64
ProgressRate              208279 non-null object
progressStatus            208279 non-null object
fileName                  208279 non-null object
time_dt                   208279 non-null datetime64[ns, pytz.FixedOffset(-300)]
dtypes: datetime64[ns, pytz.FixedOffset(-300)](1), float64(3), int64(1)

In [14]:
pd.concat([df_selected.head(), df_selected.tail()])

,LineRef,DirectionRef,DatedVehicleJourneyRef,DataFrameRef,JourneyPatternRef,OriginRef,DestinationRef,DestinationName,RecordedAtTime,Latitude,Longitude,Bearing,ProgressRate,progressStatus,fileName,time_dt
0,MTA NYCT_Q16,1,MTA NYCT_CS_A9-Weekday-SDon-045000_Q16_5,2019-02-12,MTA_Q160039,MTA_504445,MTA_804050,FLUSHING MAIN ST STATION via UTOPIA,2019-02-12T07:30:00.000-05:00,40.790069,-73.781871,253.178590,normalProgress,None,BusData_20190212_073000.xml,2019-02-12 07:30:00-05:00
1,MTA NYCT_M42,1,MTA NYCT_MQ_A9-Weekday-SDon-044900_M42_5,2019-02-12,MTA_M420134,MTA_403249,MTA_903082,42 ST PIER CROSSTOWN,2019-02-12T07:30:01.000-05:00,40.748384,-73.969951,53.841816,normalProgress,None,BusData_20190212_073000.xml,2019-02-12 07:30:01-05:00
2,MTABC_QM34,1,MTABC_22501199-LGPA9-LG_A9-Weekday-10,2019-02-12,MTA_QM340039,MTA_504366,MTA_405403,EAST MIDTOWN 55 ST via 3 AV,2019-02-12T07:30:01.000-05:00,40.701709,-73.894871,108.928600,normalProgress,None,BusData_20190212_073200.xml,2019-02-12 07:30:01-05:00
3,MTA NYCT_Q43,1,MTA NYCT_QV_A9-Weekday-SDon-042600_Q43_7,2019-02-12,MTA_Q430188,MTA_502085,MTA_904095,LTD JAMAICA LIRR STA via HILLSIDE,2019-02-12T07:30:02.000-05:00,40.713097,-73.782032,196.189210,normalProgress,None,BusData_20190212_073200.xml,2019-02-12 07:30:02-05:00
4,MTA NYCT_Q31,1,MTA NYCT_CS_A9-Weekday-SDon-039500_MISC_740,2019-02-12,MTA_Q310171,MTA_501249,MTA_904095,JAMAICA LIRR,2019-02-12T07:30:02.000-05:00,40.700268,-73.808673,291.037500,normalProgress,None,BusData_20190212_073000.xml,2019-02-12 07:30:02-05:00
208274,MTABC_Q64,1,MTABC_22560192-BPPA9-BP_A9-Weekday-10-SDon,2019-02-12,MTA_Q640095,MTA_551840,MTA_982087,FOREST HILLS 71 AV STA via JEWEL AV via 69 RD,2019-02-12T09:29:59.000-05:00,40.731787,-73.805534,177.108730,noProgress,layover,BusData_20190212_093000.xml,2019-02-12 09:29:59-05:00
208275,MTA NYCT_B38,1,MTA NYCT_GA_A9-Weekday-SDon-052400_B38_224,2019-02-12,MTA_B380161,MTA_503707,MTA_901070,DNTWN BKLYN TILLARY ST,2019-02-12T09:29:59.000-05:00,40.689975,-73.981907,181.169140,normalProgress,None,BusData_20190212_093000.xml,2019-02-12 09:29:59-05:00
208276,MTA NYCT_M101,1,MTA NYCT_OH_A9-Weekday-SDon-044600_M101_23,2019-02-12,MTA_M1010381,MTA_400448,MTA_903148,LTD EAST VILLAGE 6 ST via LEX AV,2019-02-12T09:29:59.000-05:00,40.737226,-73.984309,233.914930,normalProgress,None,BusData_20190212_093000.xml,2019-02-12 09:29:59-05:00
208277,MTA NYCT_M57,1,MTA NYCT_MQ_A9-Weekday-SDon-054100_M57_7,2019-02-12,MTA_M570101,MTA_404938,MTA_400854,WEST SIDE BWAY - 72 ST XTOWN,2019-02-12T09:29:59.000-05:00,40.766664,-73.982730,156.614780,normalProgress,None,BusData_20190212_093000.xml,2019-02-12 09:29:59-05:00
208278,MTABC_Q111,1,MTABC_22561224-BPPA9-BP_A9-Weekday-10-SDon,2019-02-12,MTA_Q1110273,MTA_552363,MTA_552285,ROSEDALE F. LEWIS BL via BREWER BL via 147 AV,2019-02-12T09:29:59.000-05:00,40.668405,-73.771856,310.055940,normalProgress,None,BusData_20190212_093000.xml,2019-02-12 09:29:59-05:00


In [15]:
columnsLists= list(df_selected.columns)
columnsLists.pop(-2)
columnsLists

['LineRef',
 'DirectionRef',
 'DatedVehicleJourneyRef',
 'DataFrameRef',
 'JourneyPatternRef',
 'OriginRef',
 'DestinationRef',
 'DestinationName',
 'RecordedAtTime',
 'Latitude',
 'Longitude',
 'Bearing',
 'ProgressRate',
 'progressStatus',
 'time_dt']

In [16]:
# The duplicated records
df_selected[df_selected.duplicated(keep = False)].sort_values(by=['DatedVehicleJourneyRef', 'RecordedAtTime'])

,LineRef,DirectionRef,DatedVehicleJourneyRef,DataFrameRef,JourneyPatternRef,OriginRef,DestinationRef,DestinationName,RecordedAtTime,Latitude,Longitude,Bearing,ProgressRate,progressStatus,fileName,time_dt


In [17]:
# The records which have the same data in the "DatedVehicleJourneyRef" and "RecordedAtTime" columns
df_selected[df_selected.duplicated(subset = ['DatedVehicleJourneyRef', 'RecordedAtTime'], keep = False)].sort_values(by=['DatedVehicleJourneyRef', 'RecordedAtTime'])

,LineRef,DirectionRef,DatedVehicleJourneyRef,DataFrameRef,JourneyPatternRef,OriginRef,DestinationRef,DestinationName,RecordedAtTime,Latitude,Longitude,Bearing,ProgressRate,progressStatus,fileName,time_dt
23857,MTA NYCT_SIM22,0,MTA NYCT_CA_E9-Weekday-SDon-039000_MISC_452,2019-02-12,MTA_SIM220039,MTA_200994,MTA_402214,MIDTOWN via 42 ST via MADISON AV,2019-02-12T07:43:46.000-05:00,40.756219,-73.976775,53.396748,normalProgress,None,BusData_20190212_074400.xml,2019-02-12 07:43:46-05:00
23909,MTA NYCT_SIM22,0,MTA NYCT_CA_E9-Weekday-SDon-039000_MISC_452,2019-02-12,MTA_SIM220039,MTA_200994,MTA_402214,MIDTOWN via 42 ST via MADISON AV,2019-02-12T07:43:46.000-05:00,40.756219,-73.976775,53.396748,normalProgress,None,BusData_20190212_074600.xml,2019-02-12 07:43:46-05:00
4857,MTA NYCT_S53,0,MTA NYCT_CA_E9-Weekday-SDon-041000_MISC_495,2019-02-12,MTA_S530127,MTA_202740,MTA_801069,BAY RIDGE 86 ST STA,2019-02-12T07:33:39.000-05:00,40.601650,-74.062705,352.874970,normalProgress,None,BusData_20190212_073600.xml,2019-02-12 07:33:39-05:00
4878,MTA NYCT_S53,0,MTA NYCT_CA_E9-Weekday-SDon-041000_MISC_495,2019-02-12,MTA_S530127,MTA_202740,MTA_801069,BAY RIDGE 86 ST STA,2019-02-12T07:33:39.000-05:00,40.601650,-74.062705,352.874970,normalProgress,None,BusData_20190212_073400.xml,2019-02-12 07:33:39-05:00
100313,MTA NYCT_SIM3,0,MTA NYCT_CA_E9-Weekday-SDon-041000_SIM3_308,2019-02-12,MTA_SIM30106,MTA_905019,MTA_903036,MIDTOWN via 23 ST via 6 AV,2019-02-12T08:25:26.000-05:00,40.765835,-73.976589,156.349900,noProgress,layover,BusData_20190212_082800.xml,2019-02-12 08:25:26-05:00
100315,MTA NYCT_SIM3,0,MTA NYCT_CA_E9-Weekday-SDon-041000_SIM3_308,2019-02-12,MTA_SIM30106,MTA_905019,MTA_903036,MIDTOWN via 23 ST via 6 AV,2019-02-12T08:25:26.000-05:00,40.765835,-73.976589,156.349900,noProgress,layover,BusData_20190212_082600.xml,2019-02-12 08:25:26-05:00
143641,MTA NYCT_SIM3,0,MTA NYCT_CA_E9-Weekday-SDon-041000_SIM3_308,2019-02-12,MTA_SIM30106,MTA_905019,MTA_903036,MIDTOWN via 23 ST via 6 AV,2019-02-12T08:49:33.000-05:00,40.765949,-73.976849,156.349900,noProgress,layover,BusData_20190212_085200.xml,2019-02-12 08:49:33-05:00
143676,MTA NYCT_SIM3,0,MTA NYCT_CA_E9-Weekday-SDon-041000_SIM3_308,2019-02-12,MTA_SIM30106,MTA_905019,MTA_903036,MIDTOWN via 23 ST via 6 AV,2019-02-12T08:49:33.000-05:00,40.765949,-73.976849,156.349900,noProgress,layover,BusData_20190212_085000.xml,2019-02-12 08:49:33-05:00
98798,MTA NYCT_S76,0,MTA NYCT_CA_E9-Weekday-SDon-041100_MISC_424,2019-02-12,MTA_S760080,MTA_203896,MTA_805015,ST GEORGE FERRY,2019-02-12T08:23:47.000-05:00,40.643585,-74.073001,199.003080,noProgress,layover,BusData_20190212_082400.xml,2019-02-12 08:23:47-05:00
98810,MTA NYCT_S76,0,MTA NYCT_CA_E9-Weekday-SDon-041100_MISC_424,2019-02-12,MTA_S760080,MTA_203896,MTA_805015,ST GEORGE FERRY,2019-02-12T08:23:47.000-05:00,40.643585,-74.073001,199.003080,noProgress,layover,BusData_20190212_082600.xml,2019-02-12 08:23:47-05:00


In [18]:
# The records which have the same data in the "DatedVehicleJourneyRef" and "filename" columns
df_selected[df_selected.duplicated(subset = ['DatedVehicleJourneyRef', 'fileName'], keep = False)].sort_values(by=['DatedVehicleJourneyRef', 'RecordedAtTime'])

,LineRef,DirectionRef,DatedVehicleJourneyRef,DataFrameRef,JourneyPatternRef,OriginRef,DestinationRef,DestinationName,RecordedAtTime,Latitude,Longitude,Bearing,ProgressRate,progressStatus,fileName,time_dt
9604,MTA NYCT_S54,0,MTA NYCT_CA_E9-Weekday-SDon-040500_S54_303,2019-02-12,MTA_S540136,MTA_203463,MTA_905106,W NW BRGHTN RICHMOND TERR,2019-02-12T07:35:49.000-05:00,40.640328,-74.117946,291.250520,normalProgress,None,BusData_20190212_073600.xml,2019-02-12 07:35:49-05:00
10698,MTA NYCT_S54,0,MTA NYCT_CA_E9-Weekday-SDon-040500_S54_303,2019-02-12,MTA_S540136,MTA_203463,MTA_905106,W NW BRGHTN RICHMOND TERR,2019-02-12T07:35:58.000-05:00,40.634073,-74.123058,4.061522,normalProgress,None,BusData_20190212_073600.xml,2019-02-12 07:35:58-05:00
131,MTA NYCT_S66,1,MTA NYCT_CA_E9-Weekday-SDon-042000_MISC_459,2019-02-12,MTA_S660058,MTA_203540,MTA_202993,PORT RICHMOND via GRYMES HILL,2019-02-12T07:31:28.000-05:00,40.626172,-74.131363,71.059140,normalProgress,None,BusData_20190212_073200.xml,2019-02-12 07:31:28-05:00
3430,MTA NYCT_S66,1,MTA NYCT_CA_E9-Weekday-SDon-042000_MISC_459,2019-02-12,MTA_S660058,MTA_203540,MTA_202993,PORT RICHMOND via GRYMES HILL,2019-02-12T07:31:58.000-05:00,40.640320,-74.130947,332.102720,noProgress,layover,BusData_20190212_073200.xml,2019-02-12 07:31:58-05:00
18072,MTA NYCT_S53,1,MTA NYCT_CA_E9-Weekday-SDon-043000_S53_305,2019-02-12,MTA_S530128,MTA_905059,MTA_905105,VICTORY BL,2019-02-12T07:39:59.000-05:00,40.610764,-74.094053,169.249040,normalProgress,None,BusData_20190212_074000.xml,2019-02-12 07:39:59-05:00
18086,MTA NYCT_S53,1,MTA NYCT_CA_E9-Weekday-SDon-043000_S53_305,2019-02-12,MTA_S530128,MTA_905059,MTA_905105,VICTORY BL,2019-02-12T07:40:01.000-05:00,40.640415,-74.131300,351.869900,noProgress,layover,BusData_20190212_074000.xml,2019-02-12 07:40:01-05:00
18576,MTA NYCT_S53,1,MTA NYCT_CA_E9-Weekday-SDon-043000_S53_305,2019-02-12,MTA_S530128,MTA_905059,MTA_905105,VICTORY BL,2019-02-12T07:41:32.000-05:00,40.612694,-74.099631,136.195760,normalProgress,None,BusData_20190212_074200.xml,2019-02-12 07:41:32-05:00
19867,MTA NYCT_S53,1,MTA NYCT_CA_E9-Weekday-SDon-043000_S53_305,2019-02-12,MTA_S530128,MTA_905059,MTA_905105,VICTORY BL,2019-02-12T07:41:43.000-05:00,40.640415,-74.131300,351.869900,noProgress,layover,BusData_20190212_074200.xml,2019-02-12 07:41:43-05:00
23220,MTA NYCT_S53,1,MTA NYCT_CA_E9-Weekday-SDon-043000_S53_305,2019-02-12,MTA_S530128,MTA_905059,MTA_905105,VICTORY BL,2019-02-12T07:43:40.000-05:00,40.614825,-74.102257,130.006260,normalProgress,None,BusData_20190212_074400.xml,2019-02-12 07:43:40-05:00
24957,MTA NYCT_S53,1,MTA NYCT_CA_E9-Weekday-SDon-043000_S53_305,2019-02-12,MTA_S530128,MTA_905059,MTA_905105,VICTORY BL,2019-02-12T07:43:55.000-05:00,40.640415,-74.131300,351.869900,noProgress,layover,BusData_20190212_074400.xml,2019-02-12 07:43:55-05:00


#### Cleaning
- Remove the "noProgress" data
- Delete one of the two records which have the same data except for filename columns.
- If there are two different records of the same time, delete both of them.
- If there are two records in a file for the same "DatedVehicleJourneyRef", keep them. If there are any problems, try to solve them in the visualization process (openFrameworks)

In [19]:
df_selected = df_selected.query("ProgressRate == 'normalProgress'")

In [20]:
df_selected = df_selected.drop_duplicates(subset=columnsLists)

In [21]:
# The records which have the same data in the "DatedVehicleJourneyRef" and "RecordedAtTime" columns
df_selected[df_selected.duplicated(subset = ['DatedVehicleJourneyRef', 'RecordedAtTime'], keep = False)].sort_values(by=['DatedVehicleJourneyRef', 'RecordedAtTime']).shape

(142, 16)

In [22]:
# The records which have the same data in the "DatedVehicleJourneyRef" and "filename" columns
df_selected[df_selected.duplicated(subset = ['DatedVehicleJourneyRef', 'fileName'], keep = False)].sort_values(by=['DatedVehicleJourneyRef', 'RecordedAtTime']).shape

(5196, 16)

In [23]:
#Remove the rows which have the same data in the "DatedVehicleJourneyRef" and "RecordedAtTime" columns
df_final = df_selected.drop_duplicates(subset = ['DatedVehicleJourneyRef', 'RecordedAtTime'], keep=False)

In [24]:
# Check if the data is removed
df_final[df_final.duplicated(subset = ['DatedVehicleJourneyRef', 'RecordedAtTime'], keep = False)].sort_values(by=['DatedVehicleJourneyRef', 'RecordedAtTime'])

,LineRef,DirectionRef,DatedVehicleJourneyRef,DataFrameRef,JourneyPatternRef,OriginRef,DestinationRef,DestinationName,RecordedAtTime,Latitude,Longitude,Bearing,ProgressRate,progressStatus,fileName,time_dt


In [25]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 187758 entries, 0 to 208278
Data columns (total 16 columns):
LineRef                   187758 non-null object
DirectionRef              187758 non-null int64
DatedVehicleJourneyRef    187758 non-null object
DataFrameRef              187758 non-null object
JourneyPatternRef         187758 non-null object
OriginRef                 187758 non-null object
DestinationRef            187758 non-null object
DestinationName           187758 non-null object
RecordedAtTime            187758 non-null object
Latitude                  187758 non-null float64
Longitude                 187758 non-null float64
Bearing                   187758 non-null float64
ProgressRate              187758 non-null object
progressStatus            187758 non-null object
fileName                  187758 non-null object
time_dt                   187758 non-null datetime64[ns, pytz.FixedOffset(-300)]
dtypes: datetime64[ns, pytz.FixedOffset(-300)](1), float64(3), int64(1)

In [26]:
def calcDif(_df, _timeOrigin):
    indexs = []
    lists = []

    for i in range(_df.shape[0]):

        indexs.append(_df.index[i])
        
        timePosTmp = (_df.iloc[i].time_dt - _timeOrigin).seconds
        if(timePosTmp < 0):
            print("Find Negative")
        
        lists.append(timePosTmp)
        
    return pd.Series(data = lists, index = indexs)    
 

In [27]:
df_final.head()

,LineRef,DirectionRef,DatedVehicleJourneyRef,DataFrameRef,JourneyPatternRef,OriginRef,DestinationRef,DestinationName,RecordedAtTime,Latitude,Longitude,Bearing,ProgressRate,progressStatus,fileName,time_dt
0,MTA NYCT_Q16,1,MTA NYCT_CS_A9-Weekday-SDon-045000_Q16_5,2019-02-12,MTA_Q160039,MTA_504445,MTA_804050,FLUSHING MAIN ST STATION via UTOPIA,2019-02-12T07:30:00.000-05:00,40.790069,-73.781871,253.178590,normalProgress,None,BusData_20190212_073000.xml,2019-02-12 07:30:00-05:00
1,MTA NYCT_M42,1,MTA NYCT_MQ_A9-Weekday-SDon-044900_M42_5,2019-02-12,MTA_M420134,MTA_403249,MTA_903082,42 ST PIER CROSSTOWN,2019-02-12T07:30:01.000-05:00,40.748384,-73.969951,53.841816,normalProgress,None,BusData_20190212_073000.xml,2019-02-12 07:30:01-05:00
2,MTABC_QM34,1,MTABC_22501199-LGPA9-LG_A9-Weekday-10,2019-02-12,MTA_QM340039,MTA_504366,MTA_405403,EAST MIDTOWN 55 ST via 3 AV,2019-02-12T07:30:01.000-05:00,40.701709,-73.894871,108.928600,normalProgress,None,BusData_20190212_073200.xml,2019-02-12 07:30:01-05:00
3,MTA NYCT_Q43,1,MTA NYCT_QV_A9-Weekday-SDon-042600_Q43_7,2019-02-12,MTA_Q430188,MTA_502085,MTA_904095,LTD JAMAICA LIRR STA via HILLSIDE,2019-02-12T07:30:02.000-05:00,40.713097,-73.782032,196.189210,normalProgress,None,BusData_20190212_073200.xml,2019-02-12 07:30:02-05:00
4,MTA NYCT_Q31,1,MTA NYCT_CS_A9-Weekday-SDon-039500_MISC_740,2019-02-12,MTA_Q310171,MTA_501249,MTA_904095,JAMAICA LIRR,2019-02-12T07:30:02.000-05:00,40.700268,-73.808673,291.037500,normalProgress,None,BusData_20190212_073000.xml,2019-02-12 07:30:02-05:00


In [28]:
tor = pd.to_datetime('2019-02-12 07:00:00-0500')
df_final['TimePos'] = calcDif(df_final, tor)

/Users/HiroyukiKondo/.pyenv/versions/anaconda3-5.2.0/envs/anaconda3-5.2.0/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [29]:
df_final = df_final.sort_values(["DatedVehicleJourneyRef", "RecordedAtTime"])
df_final = df_final.reset_index().drop("index", axis = 1)

In [30]:
df_final.head()

,LineRef,DirectionRef,DatedVehicleJourneyRef,DataFrameRef,JourneyPatternRef,OriginRef,DestinationRef,DestinationName,RecordedAtTime,Latitude,Longitude,Bearing,ProgressRate,progressStatus,fileName,time_dt,TimePos
0,MTA NYCT_SIM22,0,MTA NYCT_CA_E9-Weekday-SDon-038000_MISC_445,2019-02-12,MTA_SIM220039,MTA_200994,MTA_402214,MIDTOWN via 42 ST via MADISON AV,2019-02-12T07:31:33.000-05:00,40.753887,-73.978481,54.360190,normalProgress,None,BusData_20190212_073200.xml,2019-02-12 07:31:33-05:00,1893
1,MTA NYCT_SIM22,0,MTA NYCT_CA_E9-Weekday-SDon-038000_MISC_445,2019-02-12,MTA_SIM220039,MTA_200994,MTA_402214,MIDTOWN via 42 ST via MADISON AV,2019-02-12T07:33:40.000-05:00,40.756775,-73.976374,55.619656,normalProgress,None,BusData_20190212_073400.xml,2019-02-12 07:33:40-05:00,2020
2,MTA NYCT_SIM22,0,MTA NYCT_CA_E9-Weekday-SDon-038000_MISC_445,2019-02-12,MTA_SIM220039,MTA_200994,MTA_402214,MIDTOWN via 42 ST via MADISON AV,2019-02-12T07:35:44.000-05:00,40.761900,-73.972634,54.223923,normalProgress,None,BusData_20190212_073600.xml,2019-02-12 07:35:44-05:00,2144
3,MTA NYCT_SIM22,0,MTA NYCT_CA_E9-Weekday-SDon-038000_MISC_445,2019-02-12,MTA_SIM220039,MTA_200994,MTA_402214,MIDTOWN via 42 ST via MADISON AV,2019-02-12T07:37:55.000-05:00,40.762155,-73.972055,337.325900,normalProgress,None,BusData_20190212_073800.xml,2019-02-12 07:37:55-05:00,2275
4,MTA NYCT_SIM22,0,MTA NYCT_CA_E9-Weekday-SDon-039000_MISC_452,2019-02-12,MTA_SIM220039,MTA_200994,MTA_402214,MIDTOWN via 42 ST via MADISON AV,2019-02-12T07:31:43.000-05:00,40.756294,-73.987574,336.999800,normalProgress,None,BusData_20190212_073200.xml,2019-02-12 07:31:43-05:00,1903


In [31]:
df_final_forSave = df_final[["LineRef","DirectionRef","DatedVehicleJourneyRef", "RecordedAtTime", "Latitude", "Longitude", "TimePos"]]

In [32]:
df_final_forSave.to_csv("data/analyzedData/20190212/Journey_s_07-30-00_e_09-30-00_normProg_for_vis_from_pandas.csv", index =False)

In [33]:
df_final_forSave.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 187758 entries, 0 to 187757
Data columns (total 7 columns):
LineRef                   187758 non-null object
DirectionRef              187758 non-null int64
DatedVehicleJourneyRef    187758 non-null object
RecordedAtTime            187758 non-null object
Latitude                  187758 non-null float64
Longitude                 187758 non-null float64
TimePos                   187758 non-null int64
dtypes: float64(2), int64(2), object(3)
memory usage: 10.0+ MB
